In [3]:
import math
import random
import copy
import string
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import sys 
sys.path.append("..") 
from algebra import GaloisField as gf
from algebra import Tropical as tr

In [ ]:
random_ranges = 97
gf_n = 97
gf.Num.set_card(gf_n)
def set_random_tropical_num_in_GF(): # generate random number in tropical field
    return tr.Num(gf.Num(random.randint(0, random_ranges - 1)))

def set_random_num_in_GF(): # generate random number in GF field
    return gf.Num(random.randint(0, random_ranges - 1))

def set_np_matrix(i, j, f_getnum = lambda: 0):# generate random matrix
    if j > 0 and i > 0:
        b = [([0] * j)for line in range(i)]
        for ii in range(i):
            for jj in range(j):
                b[ii][jj] = f_getnum()
    b = np.array(b)
    return b

def convertTroArray(matrix_A: np.array) -> np.array:
    for cnti, i in enumerate(matrix_A):
        for cntj, j in enumerate(i):
            matrix_A[cnti][cntj] = tr.Num(gf.Num(j))
    matrix_A = np.array(matrix_A)
    return matrix_A


In [ ]:
def affine_function(var_vector: np.array, affine_coe: np.array):
    return np.dot(var_vector, affine_coe)

def trapdoor_quadratic_function(var_vector: np.array, quadratic_coe_list: list, linear_coe_list: list, oilNum: int):
    assert len(quadratic_coe_list) == oilNum
    y = []
    for cnt in range(oilNum):
        internal = np.dot(var_vector, quadratic_coe_list[cnt])
        y_quadratic = np.dot(internal, np.transpose(var_vector))
        y_linear = np.dot(var_vector, linear_coe_list[cnt])
        y_quadratic = y_quadratic[0][0]
        y_linear = y_linear[0][0]
        y.append(y_quadratic + y_linear)
    return np.array([y])

    

In [ ]:
def verify_ov_scheme(var_vector, affine_coe_1, quadratic_coe_list: list, linear_coe_list: list, affine_coe_2 = None, oilNum = 3):
    T = affine_function(var_vector, affine_coe_1)
    #print('T:\n{0}\n'.format(T))
    F = trapdoor_quadratic_function(T, quadratic_coe_list, linear_coe_list, oilNum)
    #print('F:\n{0}\n'.format(F))
    if affine_coe_2 is not None:
        #print(np.shape(F))
        S = affine_function(F, affine_coe_2)
        return S
    return F

def generate_random_affine_coe(varNum, gf_n):
    b = [([0] * varNum) for line in range(varNum)]
    for i in range(varNum):
        for j in range(varNum):
            b[i][j] = tr.Num(gf.Num(random.randint(0, gf_n - 1)))
    b = np.array(b)
    return b    

def generate_random_quadratic_coe(varNum, oilNum, gf_n):
    m = generate_random_affine_coe(varNum, gf_n)
    for i in range(varNum):
        for j in range(varNum):
            if j > varNum - oilNum -1 and i > varNum - oilNum -1:
                m[i][j] = tr.Num(gf.Num(0))
    return m            
            
def generate_random_vector(varNum, gf_n):
    n = [[tr.Num(gf.Num(random.randint(0, gf_n - 1))) for i in range(varNum)]]
    n = np.array(n)
    return n

In [ ]:
oilNum = 2
varNum = 4
gf.Num.card(97)
affine_coe_1 = generate_random_affine_coe(varNum, gf_n)
quadratic_coe_list = [generate_random_quadratic_coe(varNum, oilNum, gf_n) for i in range(oilNum)]
linear_coe_list = [np.transpose(generate_random_vector(varNum, gf_n)) for i in range(oilNum)]
var_vector = generate_random_vector(varNum, gf_n)
verification = verify_ov_scheme(var_vector, affine_coe_1, quadratic_coe_list, linear_coe_list, oilNum = oilNum)
print('vars:{0}\n'.format(var_vector))
print('affine_coe_1:{0}\n'.format(affine_coe_1))
print('quadratic_coe_list:{0}\n'.format(quadratic_coe_list))
print('linear_coe_list:{0}\n'.format(linear_coe_list))
print('verification:{0}\n'.format(verification))

In [ ]:
def generate_random_verification(varNum, oilNum, gf_n, is_affine_twice = False):
    affine_coe_1 = generate_random_affine_coe(varNum, gf_n)
    quadratic_coe_list = [generate_random_quadratic_coe(varNum, oilNum, gf_n) for i in range(oilNum)]
    linear_coe_list = [np.transpose(generate_random_vector(varNum, gf_n)) for i in range(oilNum)]
    var_vector = generate_random_vector(varNum, gf_n)
    if is_affine_twice == False:
        verification = verify_ov_scheme(var_vector, affine_coe_1, quadratic_coe_list, linear_coe_list, oilNum = oilNum)
    else:
        affine_coe_2 = generate_random_affine_coe(oilNum, gf_n)
        verification = verify_ov_scheme(var_vector, affine_coe_1, quadratic_coe_list, linear_coe_list, affine_coe_2, oilNum = oilNum)
    return verification

In [ ]:
def pic_set_point(img,axis):
    pixdata = img.load()
    w, h = img.size
    #print([int(l+axis[0]),int(l-axis[1])])
    if pixdata[int(axis[0]), int(h - 1 - axis[1])] < 255:
        pixdata[int(axis[0]), int(h - 1 - axis[1])] += 1
    return img


In [ ]:
# 对所有随机生成系数矩阵进行统计
oilNum = 3
varNum = 6
gf.Num.card(97)
times = 100000
w , h = (200, 200)
p = Image.new('L', (w, h), 0)
for cnt in range(times):
    (x, y, z) = generate_random_verification(varNum, oilNum, gf_n).reshape(-1)
    x = x - z
    y = y - z
    p = pic_set_point(p, (x.val.val, y.val.val))

plt.subplot(1,2,1)
plt.set_cmap('binary')
plt.imshow(p)

pixdata=p.load()
for i in range(w):
    for j in range(h):
        if pixdata[i, j] > 0:
            pixdata[i, j] = 255
plt.subplot(1,2,2)
plt.imshow(p)

In [ ]:
# 对给定的系数矩阵进行统计
oilNum = 3
varNum = 6
gf.Num.card(97)
times = 100000
w , h = (200, 200)
p = Image.new('L', (w, h), 0)
affine_coe_1 = generate_random_affine_coe(varNum, gf_n)
quadratic_coe_list = [generate_random_quadratic_coe(varNum, oilNum, gf_n) for i in range(oilNum)]
linear_coe_list = [np.transpose(generate_random_vector(varNum, gf_n)) for i in range(oilNum)]
for cnt in range(times):
    var_vector = generate_random_vector(varNum, gf_n)
    (x, y, z) = verify_ov_scheme(var_vector, affine_coe_1, quadratic_coe_list, linear_coe_list, oilNum = oilNum).reshape(-1)
    x = x - z
    y = y - z
    p = pic_set_point(p, (x.val.val, y.val.val))

plt.subplot(1,2,1)
plt.set_cmap('binary')
plt.imshow(p)

pixdata=p.load()
for i in range(w):
    for j in range(h):
        if pixdata[i, j]>0:
            pixdata[i, j]=255
plt.subplot(1,2,2)
plt.imshow(p)
print('affine_coe_1:{0}\n'.format(affine_coe_1))
print('quadratic_coe_list:{0}\n'.format(quadratic_coe_list))
print('linear_coe_list:{0}\n'.format(linear_coe_list))
